##### Titulo: Algoritmo no supervisado K-Means
#####  Autor: Dr. Gabriel Guerrero
##### Fecha: 20190704



## ¿Qué es?
**K-Means** es un algoritmo no supervisado de clasificación (clusterización).

K-Means es un algoritmo que encuentra k-grupos (k-clusters) en un conjunto de datos de forma automatica, 

El número de k de clusters o agrupaciones se define al inicio. 

Cada cluster o agrupación se describe por medio de un punto denominado **centroide**

El centroide se encuentra en el centro de todos los puntos del cluster

Este algoritmo agrupa objetos en k grupos basándose en la similitud de sus características, es decir que tan semejantes son, según la noción de distancia 

Con el algoritmo K-Means no supervisado de clasificacion se intenta agrupar cosas similares en un cluster y cosas diferentes en otros clusters 

El agrupamiento se realiza minimizando la suma de distancias entre cada objeto y el centroide de su grupo o cluster. 

Generalmente se suele usar la distancia cuadrática.

## ¿Cómo funciona?


Primero se selecciona al azar k-puntos del conjunto de datos (dataset), que se denominan los k-centroides iniciales

Enseguida, para cada punto del conjunto de datos, se busca el k-centroide mas cercano y se asigna el punto a ese cluster.

Una vez que hemos asignado todos los puntos del conjunto de datos a un cluster, para los puntos del cluster se saca el valor medio y este valor medio se define como el centroide.



El algoritmo consta de tres pasos:

- **Inicialización**: una vez escogido el número de grupos, k, se establecen k centroides en el espacio de los datos, por ejemplo, escogiéndolos aleatoriamente.
- **Asignación objetos a los centroides**: cada objeto de los datos es asignado a su centroide más cercano.
- **Actualización centroides**: se actualiza la posición del centroide de cada grupo tomando como nuevo centroide la posición del promedio de los objetos pertenecientes a dicho grupo.

Se repiten los pasos 2 y 3 hasta que los centroides no se mueven, o se mueven por debajo de una distancia umbral en cada paso.

 <img src="kmeans1.png"/>


## Ventajas

- Es un método sencillo y rápido. Pero es necesario decidir el valor de k y el resultado final depende de la inicialización de los centroides. En principio no converge al mínimo global sino a un mínimo local.


In [ ]:

#Importar bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
!head -3 pokemon.csv

In [ ]:
#Leer el archivo pokemon.csv
dfPokemon = pd.read_csv('pokemon.csv')
dfPokemon.sample(10) #Tomar una muestra aleatoria de 10 entradas 

In [ ]:
#Mostramos el tipo de entrada de cada columna
dfPokemon.info()

In [ ]:
# Llenamos los campos nulos de la columna 'Type 2' con el valor 'No_Tiene'
dfPokemon['Type 2'] = dfPokemon['Type 2'].fillna('No_Tiene')
dfPokemon.sample(10)

In [ ]:
#Construimos un nuevo DataFrame usando las columnas 'Attack', 'Defense' y 'Speed'
pokecar = dfPokemon[['HP','Attack', 'Defense', 'Speed']]
pokecar.sample(10)

In [ ]:
#Normalizamos los datos
sc_X = StandardScaler()
dataset_num = sc_X.fit_transform(pokecar)
datosestandar = pd.DataFrame(dataset_num, columns=['HP','Attack', 'Defense', 'Speed'])
datosestandar.sample(10)


## Elección del número de clusters
Aunque no exista un criterio para la selección del número de Clusters, existen diferentes métodos que nos ayudan a elegir un número apropiado de Clusters para agrupar los datos; uno de ellos es el ** método del codo (Elbow Method)**.

Este método utiliza los valores de la inercia obtenidos tras aplicar el K-means a diferente número de Clusters (desde 1 a N Clusters), siendo la inercia la suma de las distancias al cuadrado de cada objeto del Cluster a su centroide: 

$$ Inercia = \displaystyle \sum_1^N ||x_i - \mu||^2$$

Una vez obtenidos los valores de la inercia tras aplicar el K-means de 1 a N Clusters, representamos en una gráfica lineal la inercia respecto del número de Clusters. 

En esta gráfica se debería de apreciar un cambio brusco en la evolución de la inercia, teniendo la línea representada una forma similar a la de un brazo y su codo. 

El punto en el que se observa ese cambio brusco en la inercia nos dirá el número óptimo de Clusters a seleccionar para ese data set; o dicho de otra manera: el punto que representaría al codo del brazo será el número óptimo de Clusters para ese data set. 


In [ ]:
#Aplicamos el "método del codo" que nos dará una aproximación del número de clústers  

wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(datosestandar)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 10), wcss)
plt.title('El método del Codo')
plt.xlabel('Número de clústers')
plt.ylabel('WCSS')
plt.xticks(np.arange(1, 10, 1.0))
plt.grid(which='major', axis='x')
plt.show()

In [ ]:
#Aplicamos el algoritmo con 2 clústers
kmeans = KMeans(n_clusters = 2, init = 'k-means++', random_state = 42)

In [ ]:
#Calculamos las etiquetas de cada entrada
y_kmeans = kmeans.fit_predict(datosestandar)
y_kmeans

In [ ]:
#Agregamos la columna de etiquetas a nuestro DataFrame
dfPokemon['numCluster'] = y_kmeans
dfPokemon.sample(10)

In [ ]:
#Graficamos los clusters en una gráfica de violín
sns.violinplot(x='numCluster', y='Total', data=dfPokemon)
plt.show()

In [ ]:
# Distribución según defensa
sns.violinplot(x='numCluster', y='Defense', data=dfPokemon)
plt.show()

In [ ]:
dfPokemon.sort_values('Defense', axis=0, ascending=False).head(20)

In [ ]:
# Distribución según velocidad
sns.violinplot(x='numCluster', y='Speed', data=dfPokemon)
plt.show()

In [ ]:
dfPokemon.sort_values('Speed', axis=0, ascending=False).head(20)

In [ ]:
# Distribución según ataque

sns.violinplot(x='numCluster', y='Attack', data=dfPokemon)
plt.show()

In [ ]:
dfPokemon.sort_values('Attack', axis=0, ascending=False).head(20)

In [ ]:
sns.violinplot(x='numCluster', y='HP', data=dfPokemon)
plt.show()

In [ ]:
dfPokemon.sort_values('HP', axis=0, ascending=False).head(20)

In [ ]:
dfPokemon[dfPokemon['numCluster'] == 1]

In [ ]:
#Predicción usando algoritmo kmeans
kmeans.predict([[20,23, 56, 89], [0,100, 40, 55]])